In [1]:
import requests
import time
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
# 1. Obtener el HTML de la página del ranking
load_dotenv()
url = os.getenv("TARGET_URL_AGRICULTURAL_LAND")
BASE_URL = os.getenv("BASE_URL")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Lanza error si la petición falla

In [6]:
# 2. Parsear HTML con BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [7]:
# 3. Buscar la tabla principal y extraer los links
table = soup.find("table")
country_links = []


if table:
    rows = table.find_all("tr")
    for row in rows:
        cell = row.find("td")
        if cell and cell.a:
            relative_link = cell.a["href"]
            full_url = f"{BASE_URL}{relative_link}"
            country_links.append(full_url)

# Convertir la lista en DataFrame
df_links = pd.DataFrame(country_links, columns=["url"])

# Guardar como archivo Parquet
df_links.to_parquet("data/country_agricultural_land_links.parquet", index=False)

In [5]:
# 4. Mostrar resultado
print("Links encontrados:", df_links.size)

Links encontrados: 189


In [3]:
# configuracion necesaria para descargar datos

sleep_duration = 5

plots_dir = "plots"

os.makedirs(plots_dir, exist_ok=True)

In [10]:
def extraer_info_desde_url(url: str) -> tuple[str, str]:
    partes = url.strip("/").split("/")
    if len(partes) < 5:
        raise ValueError("URL no tiene el formato esperado.")
    pais = partes[-2]
    indicador = partes[-1]
    return pais, indicador

In [8]:
# Cargar los países válidos desde el archivo parquet
df_countries = pd.read_parquet("data/countries_with_hpi.parquet")
valid_countries = set(df_countries["country"].unique())

In [9]:
# logic and execution

start_time = time.time()

for url in country_links:
    pais, indicador = extraer_info_desde_url(url)

    if pais not in valid_countries:
        print(f"⏭️ País {pais} no está en la lista, se omite.")
        continue

    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        img_tag = soup.find("img", id="graphImage")
        if not img_tag:
            print(f"❌ No se encontró la imagen para {pais} - {indicador}")
            continue

        img_url = img_tag["src"]
        if img_url.startswith("/"):
            img_url = base_url + img_url

        # Crear carpeta del país dentro de "plots"
        country_folder = os.path.join(plots_dir, pais)
        os.makedirs(country_folder, exist_ok=True)

        # Guardar imagen
        filename = f"{indicador}_{pais}.png"
        filepath = os.path.join(country_folder, filename)

        img_response = requests.get(img_url, headers=headers)
        with open(filepath, "wb") as f:
            f.write(img_response.content)

        print(f"✅ Imagen guardada en: {filepath}")

    except Exception as e:
        print(f"❌ Error procesando {url}: {e}")

    time.sleep(sleep_duration)

# Medición final
end_time = time.time()
total_sleep_time = len(valid_countries) * sleep_duration
effective_time = end_time - start_time - total_sleep_time

print(f"\n⏳ Tiempo total (incluyendo esperas): {end_time - start_time:.2f} s")
print(f"😴 Tiempo en espera: {total_sleep_time:.2f} s")
print(f"⚡ Tiempo efectivo de ejecución: {effective_time:.2f} s")

⏭️ País Turkmenistan no está en la lista, se omite.
⏭️ País Ivory-Coast no está en la lista, se omite.
⏭️ País Burundi no está en la lista, se omite.
⏭️ País Rwanda no está en la lista, se omite.
⏭️ País Uruguay no está en la lista, se omite.
⏭️ País Saudi-Arabia no está en la lista, se omite.
⏭️ País Syria no está en la lista, se omite.
⏭️ País Lesotho no está en la lista, se omite.
✅ Imagen guardada en: plots\South-Africa\Percent_agricultural_land_South-Africa.png
⏭️ País Kazakhstan no está en la lista, se omite.
⏭️ País Nigeria no está en la lista, se omite.
⏭️ País Djibouti no está en la lista, se omite.
⏭️ País Bangladesh no está en la lista, se omite.
⏭️ País Mongolia no está en la lista, se omite.
⏭️ País Uganda no está en la lista, se omite.
⏭️ País Comoros no está en la lista, se omite.
⏭️ País Ukraine no está en la lista, se omite.
⏭️ País Somalia no está en la lista, se omite.
⏭️ País Madagascar no está en la lista, se omite.
⏭️ País Togo no está en la lista, se omite.
⏭️ Pa

In [8]:
# obtain paraguay url
template_url = country_links[0]
country = "Paraguay"

# Separar por "/"
parts = template_url.strip("/").split("/")

# Reemplazar el país (posición 3: después del dominio)

parts[3] = country

# Reconstruir la URL
custom_url = "/".join(parts) + "/"

In [11]:
pais, indicador = extraer_info_desde_url(custom_url)

try:
    response = requests.get(custom_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    img_tag = soup.find("img", id="graphImage")
    if not img_tag:
        print(f"❌ No se encontró la imagen para {pais} - {indicador}")
    else:
        img_url = img_tag["src"]
        if img_url.startswith("/"):
            img_url = base_url + img_url

        # Crear carpeta del país dentro de "plots"
        country_folder = os.path.join(plots_dir, pais)
        os.makedirs(country_folder, exist_ok=True)

        # Guardar imagen
        filename = f"{indicador}_{pais}.png"
        filepath = os.path.join(country_folder, filename)

        img_response = requests.get(img_url, headers=headers)
        with open(filepath, "wb") as f:
            f.write(img_response.content)

        print(f"✅ Imagen guardada en: {filepath}")

except Exception as e:
    print(f"❌ Error procesando {custom_url}: {e}")

✅ Imagen guardada en: plots\Paraguay\Percent_agricultural_land_Paraguay.png
